In [1]:
import pandas as pd
import networkx as nx
from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
ls

Centrality.ipynb                          SOL_2020-10-01_2020-12-31_TRX.csv
SOL_2020-03-19_2020-06-30_TRX.csv         SOL_Transfer_extraction_code.ipynb
SOL_2020-03-19_2020-06-30_centrality.csv  calculate_components.ipynb
SOL_2020-07-01_2020-09-30_TRX.csv         calculate_components2.ipynb


In [13]:
file_name = 'SOL_2020-10-01_2020-12-31_TRX.csv'

df = pd.read_csv(file_name)
df

,timestamp,block_height,sender_address,receiver_address,amount,trx_hash,trx_gasPrice
0,2020-10-01 00:00:26,38521310,FrDcCb3DS3nFJ41jG3kvnsKuwjJf8qmWa89QcL4Jt4MA,8ZLcmKPxQ3151kNB5yXXYbFHnKLdkoLsEP4FtUWXgWkv,3500.0000,C4Ej7RBWkzEuGZuh15szWUjHStDm6AJaQB5uM3NNdPdW,0.000005
1,2020-10-01 00:00:26,38521311,FrDcCb3DS3nFJ41jG3kvnsKuwjJf8qmWa89QcL4Jt4MA,8ZLcmKPxQ3151kNB5yXXYbFHnKLdkoLsEP4FtUWXgWkv,1000.0000,9Mtexf1LnkJVw46aTNuevsFdMW4q8nPqC57PpmrN5brB,0.000005
2,2020-10-01 00:00:27,38521312,BzZeRXC4v3fqddMBYKLSP4Awswp5eSP3qUy9y9HsgXrG,8ZLcmKPxQ3151kNB5yXXYbFHnKLdkoLsEP4FtUWXgWkv,300.0000,4JS4YcX4p81w51DjCHicERLnupD2Ek2FWpnvbrDwA7tH,0.000005
3,2020-10-01 00:00:27,38521313,BzZeRXC4v3fqddMBYKLSP4Awswp5eSP3qUy9y9HsgXrG,8ZLcmKPxQ3151kNB5yXXYbFHnKLdkoLsEP4FtUWXgWkv,4000.0000,5xcBTif5PWsiVpEkXNHSsZPnLhg77apt3kcXJ7LoqYvg,0.000005
4,2020-10-01 00:00:28,38521315,6gfi6GSjrhqc5xDLtDkVrTR61Hi7GMNPmJknxvbqzb1x,6Mk9P4C77EskX51ov95z6RnFZ8jCpn7tbDVfeeTQbr28,1234.5886,HPfPhCfz9PpmydEpHD9eh3LvYDscKaHXVrUCJ313sbq1,0.000010
...,...,...,...,...,...,...,...
4513120,2020-12-31 23:59:17,58456725,8ZLcmKPxQ3151kNB5yXXYbFHnKLdkoLsEP4FtUWXgWkv,FrDcCb3DS3nFJ41jG3kvnsKuwjJf8qmWa89QcL4Jt4MA,2000.0000,5zHfWZc7YJrxCvfp2qGuaZsCfMSWhdctoujDWafpWyTS,0.000005
4513121,2020-12-31 23:59:17,58456725,FrDcCb3DS3nFJ41jG3kvnsKuwjJf8qmWa89QcL4Jt4MA,8ZLcmKPxQ3151kNB5yXXYbFHnKLdkoLsEP4FtUWXgWkv,2000.0000,5zHfWZc7YJrxCvfp2qGuaZsCfMSWhdctoujDWafpWyTS,0.000005
4513122,2020-12-31 23:59:44,58456788,8ZLcmKPxQ3151kNB5yXXYbFHnKLdkoLsEP4FtUWXgWkv,FrDcCb3DS3nFJ41jG3kvnsKuwjJf8qmWa89QcL4Jt4MA,1800.0000,CX5mVZBsa5N3xPomE9kJ7Lu7avk9nmAfJYuULhDJJ6CL,0.000005
4513123,2020-12-31 23:59:48,58456799,8ZLcmKPxQ3151kNB5yXXYbFHnKLdkoLsEP4FtUWXgWkv,GKgXWmK8bC9CjeHrpbGR6Tv9fNQYVJsLT5QpuaHzfoHH,300.0000,Aco6J6UVLGqPhcdKHQsWN6pzJwN1owHu17GcYjn9HXj,0.000005


In [14]:
# timestamp를 datetime으로 변환
df['timestamp'] = pd.to_datetime(df['timestamp'])

# 시작일과 종료일 설정
start_date = df['timestamp'].min().date()
end_date = df['timestamp'].max().date()

# 결과를 저장할 리스트
results = []

# 진행 상황을 표시할 tqdm 객체 생성
total_days = (end_date - start_date).days + 1
pbar = tqdm(total=total_days, desc="Processing days")

# 각 날짜별로 계산
current_date = start_date
while current_date <= end_date:
    # 해당 날짜의 데이터만 필터링
    daily_df = df[df['timestamp'].dt.date == current_date]
    
    # 그래프 생성
    G = nx.from_pandas_edgelist(daily_df, 'sender_address', 'receiver_address', edge_attr='amount', create_using=nx.DiGraph())
    G.remove_edges_from(nx.selfloop_edges(G))
    
    # 그래프가 비어있지 않은 경우에만 계산
    if G.number_of_nodes() > 0:
        # Core decomposition
        core_numbers = nx.core_number(G)
        max_core = max(core_numbers.values()) if core_numbers else 0
        
        try:
            adhesion = nx.edge_connectivity(G)
        except nx.NetworkXError:
            adhesion = 0  # For disconnected graphs
        
        # Cohesion (node connectivity)
        try:
            cohesion = nx.node_connectivity(G)
        except nx.NetworkXError:
            cohesion = 0  # For disconnected graphs
        
        # 결과 저장
        results.append({
            'date': current_date,
            'max_core_number': max_core,
            'adhesion': adhesion,
            'cohesion': cohesion
        })
    
    # 다음 날짜로 이동
    current_date += timedelta(days=1)

    # 진행 상황 업데이트
    pbar.update(1)

# 진행 바 닫기
pbar.close()

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(results)

# 결과 출력
print(result_df)

Processing days: 100%|███████████████████████████████████████████████████████████████| 92/92 [02:12<00:00,  1.44s/it]

          date  max_core_number  adhesion  cohesion
0   2020-10-01                4         0         0
1   2020-10-02                4         0         0
2   2020-10-03                4         0         0
3   2020-10-04                4         0         0
4   2020-10-05                4         0         0
..         ...              ...       ...       ...
87  2020-12-27                3         0         0
88  2020-12-28                3         0         0
89  2020-12-29                3         0         0
90  2020-12-30                3         0         0
91  2020-12-31                3         0         0

[92 rows x 4 columns]


In [15]:
result_df

,date,max_core_number,adhesion,cohesion
0,2020-10-01,4,0,0
1,2020-10-02,4,0,0
2,2020-10-03,4,0,0
3,2020-10-04,4,0,0
4,2020-10-05,4,0,0
...,...,...,...,...
87,2020-12-27,3,0,0
88,2020-12-28,3,0,0
89,2020-12-29,3,0,0
90,2020-12-30,3,0,0


In [16]:
result_df.to_csv(f'{file_name[:-8]}_2.csv')

In [7]:
# 2021년 2차 계산항목 계산중